In [1]:
# Let's start with a classic; the "Monty Hall problem".
# The show's guest has three doors to choose from: A, B & C.
# So, the probability that a prize is behind any of these
# doors is P(A) = P(B) = P(C) = 1/3.

# Monty will not open the door with the prize behind it, nor
# will he open the door immediately chose by the contestant.
# That leaves two possible doors. From the contestant's 
# perspective, it's equally likely for Monty to choose
# one over the other. Thus, the marginal likelyhood is
# P(E) = 1/2.

# Our hypothesis is threefold: we're interested in the
# probability of the prize being behind a given door
# as each door is opened by Monty. 

# Hypothesis A: The prize is behind the first door. The 
# "prior" of this problem is the probability of the prize 
# being behind this door, absent any other data we have.
# P(A) = 1/3. 

# The marginal likelihood is the probability that
# Monty chooses a given door irrespective of any other 
# data. As stated above, that is P(E) = 1/2.

# The likelihood that Monty chooses the third door given
# that the prize is behind the first door is 1/2. P(E|A) = 1/2

# Now we can calculate the posterior:
# P(A|E) = (P(E|A) * P(A)) / P(E) = (1/2 * 1/3) / (1/2) = 1/3.

# Hypothesis B: 
# The marginal likelihood is the same as before. The prior is still
# 1/3. However, the likelihood has changed. If we assume that the 
# prize is behind the second door, Monty has no choice but to open 
# the third door (since he'll never open the door with the prize).
# Thus P(E|B) = 1

# Our posterior is thus:
# P(B|E) = (P(E|B) * P(B)) / P(E) = (1 * 1/3) / (1/2) = 2/3.

# In terms of Bayesian inference, we can state that our belief in 
# this hypothesis is stronger than our belief in A (since 1/3 is 
# less the 2/3). Let's see if this can be programmatically derived.

# Now, to program....!!!


In [2]:
import pomegranate as pg

In [3]:
DISTRIBUTION_CONST = 1./3
guest_door = pg.DiscreteDistribution({'A': DISTRIBUTION_CONST,
                                      'B': DISTRIBUTION_CONST,
                                      'C': DISTRIBUTION_CONST})
prize_door = pg.DiscreteDistribution({'A': DISTRIBUTION_CONST,
                                      'B': DISTRIBUTION_CONST,
                                      'C': DISTRIBUTION_CONST})

In [4]:
monty_door = pg.ConditionalProbabilityTable(
        [['A', 'A', 'A', 0.0],
         ['A', 'A', 'B', 0.5],
         ['A', 'A', 'C', 0.5],
         ['A', 'B', 'A', 0.0],
         ['A', 'B', 'B', 0.0],
         ['A', 'B', 'C', 1.0],
         ['A', 'C', 'A', 0.0],
         ['A', 'C', 'B', 1.0],
         ['A', 'C', 'C', 0.0],
         ['B', 'A', 'A', 0.0],
         ['B', 'A', 'B', 0.0],
         ['B', 'A', 'C', 1.0],
         ['B', 'B', 'A', 0.5],
         ['B', 'B', 'B', 0.0],
         ['B', 'B', 'C', 0.5],
         ['B', 'C', 'A', 1.0],
         ['B', 'C', 'B', 0.0],
         ['B', 'C', 'C', 0.0],
         ['C', 'A', 'A', 0.0],
         ['C', 'A', 'B', 1.0],
         ['C', 'A', 'C', 0.0],
         ['C', 'B', 'A', 1.0],
         ['C', 'B', 'B', 0.0],
         ['C', 'B', 'C', 0.0],
         ['C', 'C', 'A', 0.5],
         ['C', 'C', 'B', 0.5],
         ['C', 'C', 'C', 0.0]], [guest_door, prize_door])

In [5]:
s1 = pg.Node(guest_door, name="guest")
s2 = pg.Node(prize_door, name="prize")
s3 = pg.Node(monty_door, name="monty")
model = pg.BayesianNetwork("Monty Hall Puzzle")
model.add_states([s1, s2, s3])
# Now add transitions
model.add_transition(s1, s3)
model.add_transition(s2, s3)
# Finalize structure of network
model.bake()

In [6]:
def get_observations(model, observation):
    beliefs = map(str, model.predict_proba(observation))
    return '\n'.join('{} \t {}'.format(state.name, belief) for state, belief in zip(model.states, beliefs))

In [7]:
# What is the probability that the game show 
# guest chooses door 'A'?
observations = {'guest': 'A'}
guest_observation = get_observations(model, observations)
print guest_observation

guest 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :1.0,
            "C" :0.0,
            "B" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
prize 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.33333333333333337,
            "C" :0.33333333333333337,
            "B" :0.33333333333333337
        }
    ],
    "name" :"DiscreteDistribution"
}
monty 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.0,
            "C" :0.5,
            "B" :0.5
        }
    ],
    "name" :"DiscreteDistribution"
}


In [8]:
# Watch what happens when Monty chooses 'B'; our guest chooses 'A'
observations = {'guest': 'A', 'monty': 'B'}
guest_monty_observations = get_observations(model, observations)
print guest_monty_observations

guest 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :1.0,
            "C" :0.0,
            "B" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
prize 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.3333333333333334,
            "C" :0.6666666666666666,
            "B" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
monty 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.0,
            "C" :0.0,
            "B" :1.0
        }
    ],
    "name" :"DiscreteDistribution"
}


In [9]:
# What happens if Monty just chooses 'B'?
observations = {'monty': 'B'}
monty_observations = get_observations(model, observations)
print monty_observations

guest 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.5,
            "C" :0.5,
            "B" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
prize 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.5,
            "C" :0.5,
            "B" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
monty 	 {
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.0,
            "C" :0.0,
            "B" :1.0
        }
    ],
    "name" :"DiscreteDistribution"
}


In [20]:
# Okay - now to the fun part; let's train the model
data = [[ 'A', 'B', 'C' ],
        [ 'A', 'A', 'C' ],
        [ 'A', 'A', 'B' ],
        [ 'A', 'A', 'A' ],
        [ 'A', 'A', 'C' ],
        [ 'B', 'B', 'B' ],
        [ 'B', 'A', 'C' ],
        [ 'C', 'C', 'A' ],
        [ 'C', 'C', 'C' ],
        [ 'C', 'C', 'C' ],
        [ 'C', 'C', 'C' ],
        [ 'C', 'B', 'A' ]]
model = model.fit(data)

/Users/blakegage/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


In [11]:
print monty_door

C	C	C	0.75
C	C	B	0.0
C	C	A	0.25
C	B	C	0.0
C	B	B	0.0
C	B	A	1.0
C	A	C	0.333333333333
C	A	B	0.333333333333
C	A	A	0.333333333333
B	C	C	0.333333333333
B	C	B	0.333333333333
B	C	A	0.333333333333
B	B	C	0.5
B	B	B	0.5
B	B	A	0.0
B	A	C	0.333333333333
B	A	B	0.333333333333
B	A	A	0.333333333333
A	C	C	0.333333333333
A	C	B	0.333333333333
A	C	A	0.333333333333
A	B	C	0.333333333333
A	B	B	0.333333333333
A	B	A	0.333333333333
A	A	C	0.6
A	A	B	0.2
A	A	A	0.2


In [12]:
print prize_door

{
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.4166666666666667,
            "C" :0.3333333333333333,
            "B" :0.25
        }
    ],
    "name" :"DiscreteDistribution"
}


In [13]:
print guest_door

{
    "frozen" :false,
    "class" :"Distribution",
    "parameters" :[
        {
            "A" :0.4166666666666667,
            "C" :0.4166666666666667,
            "B" :0.16666666666666666
        }
    ],
    "name" :"DiscreteDistribution"
}
